# TFM Cambio Climático

### Importaciones necesarias

In [ ]:
import pandas as pd
import numpy as np
import pymysql
from pymysql.constants import CLIENT
from dotenv import load_dotenv
import os

### Cargar las variables de entorno desde el archivo .env


In [ ]:
load_dotenv()

# Obtener los parámetros de conexión
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

### DataSet Original

In [ ]:
ruta = './data/Environment_Temperature_change_E_All_Data_NOFLAG.csv'
df_original = pd.read_csv(ruta, encoding='latin-1', sep=',')
print('Cabezera del DataFrame original: ')
print(df_original.head())
print('Dimensiones del DataFrame original: ')
print(df_original.info())

### Valores Nulos del Dataset

In [ ]:
print(df_original.isnull().sum().to_string())


In [ ]:
nulls_by_country_total = df_original.groupby('Area').apply(lambda group: group.isnull().sum().sum())
nulls_with_data = nulls_by_country_total[nulls_by_country_total > 0]
print(nulls_with_data.to_string())


In [ ]:

# Definir las columnas de años (comienzan con 'Y')
year_columns = [col for col in df_original.columns if col.startswith('Y')]

# Definir filtros según "Months Code"
# Mensual: 7001 - 7012 
mensual_filter = (df_original['Months Code'] >= 7001) & (df_original['Months Code'] <= 7012)
# Trimestral: 7016 - 7019
trimestral_filter = (df_original['Months Code'] >= 7016) & (df_original['Months Code'] <= 7019)
# Anual: Código 7020
anual_filter = (df_original['Months Code'] == 7020)

# Filtros por tipo de parámetro
derivacion_filter = (df_original['Element'] == 'Standard Deviation')
cambio_filter     = (df_original['Element'] == 'Temperature change')

# 4. Función para transformar el DataFrame: filtrar y pivotar (melt)
def crear_dataset(filtro_periodo, filtro_param, df_original):
    # Filtrar según período y tipo de parámetro
    df_temp = df_original[filtro_periodo & filtro_param].copy()
    # Pivotar las columnas de año a formato largo
    df_temp = pd.melt(df_temp,
                      id_vars=['Area Code', 'Area', 'Months Code','Unit'],
                      value_vars=year_columns,
                      var_name='Year',
                      value_name='Value')
    # Convertir la columna Year, quitando la 'Y' y convirtiéndola a entero
    df_temp['Year'] = df_temp['Year'].str.replace('Y', '').astype(int)
    return df_temp

# 5. Crear los 6 datasets

# Derivación estandar
derivacion_estandar_mensual    = crear_dataset(mensual_filter,  derivacion_filter, df_original)
derivacion_estandar_trimestral = crear_dataset(trimestral_filter, derivacion_filter, df_original)
derivacion_estandar_anual      = crear_dataset(anual_filter,     derivacion_filter, df_original)

# Cambio de temperatura
cambio_temperatura_mensual     = crear_dataset(mensual_filter,  cambio_filter, df_original)
cambio_temperatura_trimestral  = crear_dataset(trimestral_filter, cambio_filter, df_original)
cambio_temperatura_anual       = crear_dataset(anual_filter,     cambio_filter, df_original)

# Ejemplo: mostrar las primeras filas del dataset derivacion_estandar_mensual
print(derivacion_estandar_mensual.head())
print(derivacion_estandar_mensual.count())
print(derivacion_estandar_trimestral.count())


In [ ]:
datasets = {
    "derivacion_estandar_mensual": derivacion_estandar_mensual,
    "derivacion_estandar_trimestral": derivacion_estandar_trimestral,
    "derivacion_estandar_anual": derivacion_estandar_anual,
    "cambio_temperatura_mensual": cambio_temperatura_mensual,
    "cambio_temperatura_trimestral": cambio_temperatura_trimestral,
    "cambio_temperatura_anual": cambio_temperatura_anual,
}

for name, df in datasets.items():
    print(f"Valores nulos para {name}:")
    # Calcular la suma de valores nulos por columna y filtrar las que tengan > 0
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0].to_string())
    print("\n" + "-"*40 + "\n")


In [ ]:
# Iterar por cada dataset y eliminar filas que contengan algún valor nulo
for name, df in datasets.items():
    filas_originales = df.shape[0]
    df_clean = df.dropna()
    df_clean = df_clean.drop_duplicates()
    filas_limpias = df_clean.shape[0]
    print(f"{name}: {filas_originales} filas originales -> {filas_limpias} filas después de eliminar nulos")
    datasets[name] = df_clean

# Si lo deseas, reasigna los datasets a las variables originales:
derivacion_estandar_mensual    = datasets["derivacion_estandar_mensual"]
derivacion_estandar_trimestral = datasets["derivacion_estandar_trimestral"]
derivacion_estandar_anual      = datasets["derivacion_estandar_anual"]
cambio_temperatura_mensual     = datasets["cambio_temperatura_mensual"]
cambio_temperatura_trimestral  = datasets["cambio_temperatura_trimestral"]
cambio_temperatura_anual       = datasets["cambio_temperatura_anual"]

## Modelado de datos

### 1. Creación de la base de datos

Hay que tener en cuenta, que se debe de crear la base de datos en mysql con el nombre de `tfm_cambio_climatico`

In [ ]:
# 1. Conectarse sin especificar base de datos para eliminar/crear el esquema.
conexion = pymysql.connect(
    host='localhost',
    user='root',
    password='admin',
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()

# Eliminar el esquema si existe y crear uno nuevo.
sql_drop_db = "DROP DATABASE IF EXISTS tfm_cambio_climatico;"
sql_create_db = "CREATE DATABASE tfm_cambio_climatico;"
cursor.execute(sql_drop_db)
conexion.commit()
cursor.execute(sql_create_db)
conexion.commit()

cursor.close()
conexion.close()

print("Esquema 'tfm_cambio_climatico' creado exitosamente.")

In [ ]:
# 2. Reconectar usando el esquema recién creado.
conexion = pymysql.connect(
    host='localhost',
    user='root',
    password='admin',
    database='tfm_cambio_climatico',
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()

# (Opcional) Eliminar las tablas si existen. Aunque la base de datos es nueva, incluimos el bloque por seguridad.
sql_drop_all = """
SET FOREIGN_KEY_CHECKS=0;
DROP TABLE IF EXISTS CambioDeTemperaturas;
DROP TABLE IF EXISTS DesviacionEstandar;
DROP TABLE IF EXISTS Periodos;
DROP TABLE IF EXISTS TipoDePeriodos;
DROP TABLE IF EXISTS Paises;
DROP TABLE IF EXISTS Unidades;
SET FOREIGN_KEY_CHECKS=1;
"""

# Dividir y ejecutar cada sentencia individualmente.
statements = [stmt.strip() for stmt in sql_drop_all.split(';') if stmt.strip()]
for stmt in statements:
    cursor.execute(stmt)
conexion.commit()

In [ ]:
# 3. Crear las tablas en el esquema nuevo.
# Tabla Paises
sql_create_paises = """
CREATE TABLE IF NOT EXISTS Paises (
    id INT AUTO_INCREMENT PRIMARY KEY,
    descripcion VARCHAR(255) NOT NULL
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_paises)


# Tabla Unidades
sql_create_unidades = """
CREATE TABLE IF NOT EXISTS Unidades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    descripcion VARCHAR(255) NOT NULL
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_unidades)

# Tabla TipoDePeriodos
sql_create_tipo_periodos = """
CREATE TABLE IF NOT EXISTS TipoDePeriodos (
    id INT PRIMARY KEY,
    descripcion VARCHAR(255) NOT NULL
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_tipo_periodos)

# Tabla Periodos
sql_create_periodos = """
CREATE TABLE IF NOT EXISTS Periodos (
    id INT PRIMARY KEY,
    tipo_periodo_id INT NOT NULL,
    descripcion VARCHAR(255) NOT NULL,
    FOREIGN KEY (tipo_periodo_id) REFERENCES TipoDePeriodos(id)
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_periodos)

# Tabla CambioDeTemperaturas
sql_create_cambio_temperaturas = """
CREATE TABLE IF NOT EXISTS CambioDeTemperaturas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    pais_id INT NOT NULL,
    periodo_id INT NOT NULL,
    anio INT NOT NULL,
    unidad_id INT NOT NULL,
    valor DECIMAL(10,6),
    FOREIGN KEY (pais_id) REFERENCES Paises(id),
    FOREIGN KEY (periodo_id) REFERENCES Periodos(id),
    FOREIGN KEY (unidad_id) REFERENCES Unidades(id)
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_cambio_temperaturas)

# Tabla DesviacionEstandar
sql_create_desviacion_estandar = """
CREATE TABLE IF NOT EXISTS DesviacionEstandar (
    id INT AUTO_INCREMENT PRIMARY KEY,
    pais_id INT NOT NULL,
    periodo_id INT NOT NULL,
    anio INT NOT NULL,
    unidad_id INT NOT NULL,
    valor DECIMAL(10,6),
    FOREIGN KEY (pais_id) REFERENCES Paises(id),
    FOREIGN KEY (periodo_id) REFERENCES Periodos(id),
    FOREIGN KEY (unidad_id) REFERENCES Unidades(id)
) ENGINE=InnoDB;
"""
cursor.execute(sql_create_desviacion_estandar)

conexion.commit()
cursor.close()
conexion.close()

print("Base de datos y tablas creadas exitosamente.")

### 2. Inserción de datos en la base de datos

In [ ]:
conexion.ping(reconnect=True)

cursor = conexion.cursor()

# Insertar en TipoDePeriodos (IDs fijos)
sql_insert_tipo = "INSERT INTO TipoDePeriodos (id, descripcion) VALUES (%s, %s);"
tipos = [
    (1, 'Anual'),
    (2, 'Trimestral'),
    (3, 'Mensual')
]
cursor.executemany(sql_insert_tipo, tipos)
conexion.commit()

# Insertar registros en la tabla Periodos
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
periodos_insert = []
for i, mes in enumerate(meses, start=1):
    periodos_insert.append((i, 3, mes))
# Para períodos trimestrales: IDs 13 a 16, tipo_periodo_id = 2, descripción = 'Trimestre 1', etc.
trimestres = ['Trimestre 1', 'Trimestre 2', 'Trimestre 3', 'Trimestre 4']
for i, tri in enumerate(trimestres, start=13):
    periodos_insert.append((i, 2, tri))
# Para período anual: ID 17, tipo_periodo_id = 1, descripción = 'Anual'
periodos_insert.append((17, 1, 'Anual'))

sql_insert_periodos = "INSERT INTO Periodos (id, tipo_periodo_id, descripcion) VALUES (%s, %s, %s);"
cursor.executemany(sql_insert_periodos, periodos_insert)
conexion.commit()


# Mapeos para Paises y Unidades:
paises = set()
unidades = set()
for df in datasets.values():
    paises.update(df['Area'].unique())
    unidades.update(df['Unit'].unique())
paises = list(paises)
unidades = list(unidades)

# Insertar Paises y generar un mapeo {pais: id}
country_mapping = {}
for pais in paises:
    sql = "INSERT INTO Paises (descripcion) VALUES (%s)"
    cursor.execute(sql, (pais,))
    country_mapping[pais] = cursor.lastrowid

# Insertar Unidades y generar un mapeo {unidad: id}
unit_mapping = {}
for unit in unidades:
    sql = "INSERT INTO Unidades (descripcion) VALUES (%s)"
    cursor.execute(sql, (unit,))
    unit_mapping[unit] = cursor.lastrowid

conexion.commit()

print("Dimensiones Paises y Unidades insertadas exitosamente.")

def get_period_id(dataset_name, months_code):
    try:
        # Convertir a entero (en caso de que venga como cadena)
        months_code = int(months_code)
    except (ValueError, TypeError):
        return None

    if "mensual" in dataset_name:
        # Ejemplo: 7001 → 1, 7002 → 2, ..., 7012 → 12
        return months_code - 7000
    elif "trimestral" in dataset_name:
        # Ejemplo: 7016 → 13, 7017 → 14, 7018 → 15, 7019 → 16
        return (months_code - 7015) + 12
    elif "anual" in dataset_name:
        return 17
    else:
        return None

# Conjunto de IDs válidos en la tabla Periodos: 1-12 (mensual), 13-16 (trimestral) y 17 (anual)
valid_period_ids = set(range(1, 13)) | set(range(13, 17)) | {17}

fact_inserts_desviacion = []
fact_inserts_cambio = []

# Recorrer cada dataset y preparar las filas para la inserción
for dataset_name, df in datasets.items():
    if "derivacion_estandar" in dataset_name:
        target_list = fact_inserts_desviacion
    elif "cambio_temperatura" in dataset_name:
        target_list = fact_inserts_cambio
    else:
        continue

    for _, row in df.iterrows():
        pais = row['Area']
        months_code = row['Months Code']
        anio = row['Year']
        unidad = row['Unit']
        valor = row['Value']
        
        period_id = get_period_id(dataset_name, months_code)
        
        
        if period_id is None or period_id not in valid_period_ids:
            print(f"Advertencia: period_id {period_id} calculado a partir de Months Code {months_code} en dataset '{dataset_name}' no es válido. Se omite la fila.")
            continue
        
        pais_id = country_mapping.get(pais)
        unidad_id = unit_mapping.get(unidad)
        
        if pais_id is None:
            print(f"Advertencia: país '{pais}' no encontrado en country_mapping. Se omite la fila.")
            continue
        if unidad_id is None:
            print(f"Advertencia: unidad '{unidad}' no encontrada en unit_mapping. Se omite la fila.")
            continue
        
        target_list.append((pais_id, period_id, anio, unidad_id, valor))

# Sentencias SQL para la inserción en las tablas de hechos
sql_insert_desviacion = """
INSERT INTO DesviacionEstandar (pais_id, periodo_id, anio, unidad_id, valor)
VALUES (%s, %s, %s, %s, %s)
"""
sql_insert_cambio = """
INSERT INTO CambioDeTemperaturas (pais_id, periodo_id, anio, unidad_id, valor)
VALUES (%s, %s, %s, %s, %s)
"""

# Insertar en las tablas de hechos
try:
    cursor.executemany(sql_insert_desviacion, fact_inserts_desviacion)
    cursor.executemany(sql_insert_cambio, fact_inserts_cambio)
    conexion.commit()
    print("Datos insertados en las tablas de hechos exitosamente.")
except pymysql.err.IntegrityError as e:
    print("Error de integridad al insertar datos:", e)
    conexion.rollback()

# Cerrar cursor y conexión
cursor.close()
conexion.close()